# Auto Correct

In this notebook I want to implement a fully working auto correct system which is simple yet functional. To elaborate, the system would correct words that are 1 and 2 edit distances away. 
An edit could consist of one of the following:
- Delete: "mangoz" -> "mango", "mngoz", ...

- Switch: "atm": "tam", "amt", ...

- Replace: "rat": "hat", "mat", "rap", ...

- Insert: "pla": "play", "pela", ...

# Data PreProcessing


In [2]:
import re
from collections import Counter
import numpy as np
import pandas as pd
import random

The first step is to define a __process_data__ function that would read the corpus --*shakespeare.txt*-- and build a vocabulary based on that.

In [3]:
def process_data(file_name):
    """
    Input: 
        A file_name which is found in your current directory. You            just have to read it in. 
    Output: 
        words: a list containing all the  words in the corpus (text           file you read) in lower case. 
    """
    with open(file_name) as f:
        raw = f.read()
    low_data = raw.lower()
    words = re.findall(r"\w+", low_data)
    
    return words

In [4]:
words = process_data("shakespeare.txt")
vocab = set(words)
print(f"There are {len(vocab)} unique words. \nHere's 10 random samples from the vocabulary:")
random.sample(vocab, 10)

There are 6116 unique words. 
Here's 10 random samples from the vocabulary:


C:\Users\hesam\AppData\Local\Temp/ipykernel_20600/4094430650.py:4: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(vocab, 10)


['sets',
 'sighs',
 'cox',
 'awards',
 'dyed',
 'physic',
 'terrors',
 'reference',
 'abhor',
 'bind']

Now, we need a dictionary with words as its keys and the value of each is the number of times that word appear in our corpus. A word_frequency dictionary in essence. 

In [5]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the           word and value is its frequency.
    '''
    
    word_count_dict = {} 
    for word in word_l:
        
        if word_count_dict.get(word, False):
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1

    return word_count_dict

In [6]:
word_frequency = get_count(words)
sample_words = random.sample(list(word_frequency),10)
{k: word_frequency[k] for k in sample_words} ## 10 random sample from word_frequency dictionary

{'travel': 7,
 'perceived': 2,
 '54': 1,
 'except': 2,
 'eclipses': 2,
 'carrier': 1,
 'sharpness': 1,
 'punishment': 1,
 'guards': 1,
 'curstness': 1}

In [7]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the           word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values          are the probability that a word will occur. 
    '''
    
    total_words = sum(word_count_dict.values())
    probs = {word: word_count_dict[word]/total_words for word in word_count_dict.keys()}
    
    return probs

In [8]:
probs = get_probs(word_frequency)

# String Edits

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 

In [9]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which you will generate all                possible words in the vocabulary which have 1 missing                character
    Output:
        delete_l: a list of all possible strings obtained by                 deleting 1 character from word
    '''
     
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    delete_l = [L+R[1:] for L,R in split_l if R]


    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [10]:
delete_word_l = delete_letter(word="brand",
                        verbose=True)

input word brand, 
split_l = [('', 'brand'), ('b', 'rand'), ('br', 'and'), ('bra', 'nd'), ('bran', 'd')], 
delete_l = ['rand', 'band', 'brnd', 'brad', 'bran']


In [11]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent           charater switched
    ''' 
    
    
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    switch_l = [L[:-1] + R[0] + L[-1] + R[1:] for L,R in split_l if R and L]

    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [12]:
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta'), ('et', 'a')] 
switch_l = ['tea', 'eat']


In [13]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    replace_l = [L + let + R[1:] for L,R in split_l for let in letters]
    replace_set = set(replace_l)
    replace_set.remove(word)

    
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [14]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [15]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_l = [(word[:i], word[i:]) for i in range(len(word)+1)]
    insert_l = [L + let + R for L,R in split_l for let in letters]


    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [16]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


# Combining Edits
We need two functions that wrap up all the possible single and double edits. `edit_one_letter()` and `edit_two_letters()`

In [17]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    edit_one_set.update(insert_letter(word))
    edit_one_set.update(replace_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    

    return edit_one_set

In [18]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


Now we want to get all the possible strings that are two edits away from a given word. This can be done by performing the `edit_one_letter` method on the set of one edits of a given word.

In [19]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
    edit_two_set = set()
    
    edit_one = edit_one_letter(word,allow_switches)
    for w in edit_one:
        if w:
            edit_two = edit_one_letter(w,allow_switches)
            edit_two_set.update(edit_two)

    
    return edit_two_set

In [20]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


# Suggest Spelling
At this point, we have a set of possible 2 edits of the word. We need to find the most probable spelling from them. A function called `get_corrections` needs to be implemented that for any given word, it would return a list of edited words and their respective probabilities

In [21]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    

    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]

    
    if verbose: print("entered word = ", word, "\nsuggestions = ", suggestions)

    return n_best

In [22]:
# Test your implementation - feel free to try other words in my word
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

entered word =  dys 
suggestions =  ['days', 'dye']
word 0: dye, probability 0.000019
word 1: days, probability 0.000410
data type of corrections <class 'list'>


You can try different typos to see how well this simple auto correct works:


In [45]:
SAMPLE_WORD = "dys"
corrected = get_corrections(SAMPLE_WORD, probs, vocab, 5, verbose=True)
max_prob = 0
ind = 0
for i, sug in enumerate(corrected):
    if sug[1] > max_prob:
        ind = i
        max_prob = sug[1]
if corrected:
    print("The most probable correction is: ", corrected[ind][0])

entered word =  dys 
suggestions =  ['days', 'dye']
The most probable correction is:  days


## Reference 
NLP Specialization course from Stanford university